In [ ]:
import tensorflow.keras as keras

from neural_collaborative_filtering.Dataset import Dataset

In [ ]:
keras.__version__

In [ ]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    num_items = train.shape[1]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [ ]:
dataset = Dataset("./neural_collaborative_filtering/Data/ml-1m")
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives

In [ ]:
epochs = 20
batch_size = 256
learning_rate = 0.001
num_negatives = 4

# for gmf
latent_dim = 8

# for mlp
embedding_size = 64

# regs = [0, 0]

# 모델 확인용.. data를 실제로 넣어서 학습할때는 없애야함.
# user_num = 100
# item_num = 100
user_num = train.shape[0]
item_num = train.shape[1]

In [ ]:
# mlp
# user, item을 임베딩한후..concat, mlp layer 통과후 prediction 추출

user_input_mlp = keras.layers.Input(shape = (1,), dtype = "int32", name = "user_input_mlp")
item_input_mlp = keras.layers.Input(shape = (1,), dtype = "int32", name = "item_input_mlp")

user_embedding_mlp = keras.layers.Embedding(input_dim = user_num,
                                            output_dim = int(embedding_size / 2),
                                            name = "user_embedding_mlp",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

item_embedding_mlp = keras.layers.Embedding(input_dim = item_num,
                                            output_dim = int(embedding_size / 2),
                                            name = "item_embedding_mlp",
                                            embeddings_initializer = keras.initializers.RandomNormal(mean = 0,
                                                                                                     stddev = 0.01),
#                                             embeddings_regularizer = 0,
                                            input_length = 1)

user_latent_mlp = keras.layers.Flatten()(user_embedding_mlp(user_input_mlp)) # 32
item_latent_mlp = keras.layers.Flatten()(item_embedding_mlp(item_input_mlp)) # 32

concat = keras.layers.concatenate([user_latent_mlp, item_latent_mlp]) # 64

mlp_1 = keras.layers.Dense(units = embedding_size / 2, activation = "relu", name = "mlp_1")(concat)
mlp_2 = keras.layers.Dense(units = embedding_size / 4, activation = "relu", name = "mlp_2")(mlp_1)
mlp_3 = keras.layers.Dense(units = embedding_size / 8, activation = "relu", name = "mlp_3")(mlp_2)

prediction_mlp = keras.layers.Dense(1,
                                    activation = "sigmoid",
                                    kernel_initializer = "lecun_uniform",
                                    name = "prediction_mlp")(mlp_3)

model_mlp = keras.models.Model(inputs = [user_input_mlp, item_input_mlp],
                               outputs = prediction_mlp)

In [ ]:
model_mlp.summary()

In [ ]:
model_mlp.compile(optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = "binary_crossentropy")

In [ ]:
user_input, item_input, labels = get_train_instances(train, num_negatives)


hist = model_mlp.fit([np.array(user_input), np.array(item_input)],
                     np.array(labels),
                     batch_size = batch_size,
                     epochs = epochs,
                     verbose = 1,
                     shuffle = True)

In [ ]:
model_mlp.save_weights(filepath = "./model/model_mlp.h5")